In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"5,919,234","+6,620","229,856",+819,"3,151,695","+4,140","2,537,683","25,709",,,,,,North America,\n,,
1,,\nSouth America\n,"4,543,273","+1,282","154,885",+80,"3,116,150",+936,"1,272,238","14,295",,,,,,South America,\n,,
2,,\nAsia\n,"4,774,359",+57,"105,261",+1,"3,587,258",+73,"1,081,840","18,785",,,,,,Asia,\n,,
3,,\nEurope\n,"2,982,576",,"205,232",,"1,793,151",,"984,193","5,200",,,,,,Europe,\n,,
4,,\nAfrica\n,"1,011,867",,"22,114",,"693,620",,"296,133","1,187",,,,,,Africa,\n,,


In [6]:
df_bs.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
212,205,Montserrat,13,,1,,10,,2,,"2,604",200,61,"12,220","4,992",North America,384,"4,992",82
213,206,Caribbean Netherlands,13,,,,7,,6,,495,,424,"16,154","26,247",North America,"2,019",,62
214,207,Falkland Islands,13,,,,13,,0,,"3,726",,"1,816","520,493","3,489",South America,268,,2
215,208,Vatican City,12,,,,12,,0,,"14,981",,,,801,Europe,67,,
216,209,Western Sahara,10,,1,,8,,1,,17,2,,,"598,682",Africa,"59,868","598,682",


In [7]:
# drop unwanted rows
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)

# drop unwanted columns
df_bs = df_bs.drop('#', axis=1)

# first few rows
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"5,032,179",,"162,804",,"2,576,668",,"2,292,707","18,296","15,194",492,"63,139,605","190,640","331,198,130",North America
1,Brazil,"2,917,562",,"98,644",,"2,047,660",,"771,258","8,318","13,716",464,"13,206,188","62,085","212,710,692",South America
2,India,"2,025,409",,"41,638",,"1,377,384",,"606,387","8,944","1,466",30,"22,149,351","16,035","1,381,344,997",Asia
3,Russia,"871,894",,"14,606",,"676,357",,"180,931","2,300","5,974",100,"29,716,907","203,623","145,940,924",Europe
4,South Africa,"538,184",,"9,604",,"387,316",,"141,264",539,"9,063",162,"3,149,807","53,044","59,381,566",Africa


In [8]:
# rename columns
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

# rearrange and subselect columns
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,198,130","5,032,179",,"162,804",,"2,576,668",,"2,292,707","18,296","15,194",492,"63,139,605","190,640"
1,Brazil,South America,"212,710,692","2,917,562",,"98,644",,"2,047,660",,"771,258","8,318","13,716",464,"13,206,188","62,085"
2,India,Asia,"1,381,344,997","2,025,409",,"41,638",,"1,377,384",,"606,387","8,944","1,466",30,"22,149,351","16,035"
3,Russia,Europe,"145,940,924","871,894",,"14,606",,"676,357",,"180,931","2,300","5,974",100,"29,716,907","203,623"
4,South Africa,Africa,"59,381,566","538,184",,"9,604",,"387,316",,"141,264",539,"9,063",162,"3,149,807","53,044"


In [9]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [10]:
# addding 'WHO Region' column
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['French Guiana', 'CAR', 'Mayotte', 'Diamond Princess', 'Réunion',
       'Channel Islands', 'Isle of Man', 'Guadeloupe', 'Martinique',
       'Faeroe Islands', 'Aruba', 'Cayman Islands', 'Gibraltar',
       'Sint Maarten', 'Brunei ', 'Turks and Caicos', 'French Polynesia',
       'St. Vincent Grenadines', 'Saint Martin', 'Macao', 'Curaçao',
       'New Caledonia', 'Montserrat', 'Caribbean Netherlands',
       'Falkland Islands'], dtype=object)

In [11]:
# fix data
for col in df_bs.columns[2:]:
    # replace comma with empty string
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_bs = df_bs.replace('', np.nan)

In [12]:
# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331198130,5032179,NaN,162804,NaN,2576668,NaN,2292707,18296,15194,492,63139605,190640,Americas
1,Brazil,South America,212710692,2917562,NaN,98644,NaN,2047660,NaN,771258,8318,13716,464,13206188,62085,Americas
2,India,Asia,1381344997,2025409,NaN,41638,NaN,1377384,NaN,606387,8944,1466,30,22149351,16035,South-EastAsia
3,Russia,Europe,145940924,871894,NaN,14606,NaN,676357,NaN,180931,2300,5974,100,29716907,203623,Europe
4,South Africa,Africa,59381566,538184,NaN,9604,NaN,387316,NaN,141264,539,9063,162,3149807,53044,Africa


In [13]:
# first few rows
df_bs.tail()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
204,Montserrat,North America,4992,13,NaN,1,NaN,10,NaN,2,NaN,2604,200,61,12220,NaN
205,Caribbean Netherlands,North America,26247,13,NaN,NaN,NaN,7,NaN,6,NaN,495,NaN,424,16154,NaN
206,Falkland Islands,South America,3489,13,NaN,NaN,NaN,13,NaN,0,NaN,3726,NaN,1816,520493,NaN
207,Vatican City,Europe,801,12,NaN,NaN,NaN,12,NaN,0,NaN,14981,NaN,NaN,NaN,Europe
208,Western Sahara,Africa,598682,10,NaN,1,NaN,8,NaN,1,NaN,17,2,NaN,NaN,Africa


In [14]:
# save as .csv file
df_bs.to_csv('worldometer_data.csv', index=False)